In [1]:
import torch
import torch.nn as nn
from torch import Tensor
from transformers import GPTNeoXForCausalLM, AutoTokenizer, AutoModelForCausalLM
from jaxtyping import Float, Int
from typing import List, Optional, Tuple, Dict
import sys
from functools import partial
from tqdm import tqdm
import json
import seaborn as sns
import pandas as pd
import multiprocessing
import pickle

from request_patching import request_patch_one_pair, create_patch_request_dict, baseline_completion, baseline_completion_plus
from models import get_model_from_name

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device =', device)
torch.set_grad_enabled(False)

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")

else:
    print("CUDA is not available. Listing CPUs instead.")
    print(multiprocessing.cpu_count())

/home/jsalle/.conda/envs/athena_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device = cpu
CUDA is not available. Listing CPUs instead.
48


In [2]:
dict = create_patch_request_dict('gpt2-small',
                                 dataset='dialogs_2')

Number of stories: 2


In [3]:
dict

{'pair_0': {'context_1': "Alice: (Sighing with relief) I finally finished that massive project I've been working on for weeks. I was so stressed about meeting the deadline, but now that it's done, I feel like a huge weight has been lifted off my shoulders. Bob: (Anxiously) That's great for you, Alice. I wish I could say the same. I just found out I have to give a presentation next week, and public speaking terrifies me. I'm not sure how I'm going to get through it. Alice: (Encouragingly) Oh, Bob, I know how daunting that can feel. But remember the last time you were nervous about presenting? You ended up doing a fantastic job. Everyone was impressed. Bob: (Still worried) Thanks, Alice. But it feels different this time. I keep imagining all the things that could go wrong. What if I freeze up or forget what to say? If I had to choose between 'angry', 'proud', and 'worried', I would say I felt '",
  'context_2': "Alice: (Fuming) You won't believe the day I've had! My presentation got move

In [4]:
dict = create_patch_request_dict('pythia-1b',
                                 dataset='dialogs_2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of stories: 2


In [5]:
dict

{'pair_0': {'context_1': "Alice: (Sighing with relief) I finally finished that massive project I've been working on for weeks. I was so stressed about meeting the deadline, but now that it's done, I feel like a huge weight has been lifted off my shoulders. Bob: (Anxiously) That's great for you, Alice. I wish I could say the same. I just found out I have to give a presentation next week, and public speaking terrifies me. I'm not sure how I'm going to get through it. Alice: (Encouragingly) Oh, Bob, I know how daunting that can feel. But remember the last time you were nervous about presenting? You ended up doing a fantastic job. Everyone was impressed. Bob: (Still worried) Thanks, Alice. But it feels different this time. I keep imagining all the things that could go wrong. What if I freeze up or forget what to say? If I had to choose between 'proud', 'angry', and 'worried', I would say I felt '",
  'context_2': "Alice: (Fuming) You won't believe the day I've had! My presentation got move

In [6]:
dict = create_patch_request_dict('pythia-2.8b',
                                 dataset='dialogs_2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of stories: 2


In [7]:
dict

{'pair_0': {'context_1': "Alice: (Sighing with relief) I finally finished that massive project I've been working on for weeks. I was so stressed about meeting the deadline, but now that it's done, I feel like a huge weight has been lifted off my shoulders. Bob: (Anxiously) That's great for you, Alice. I wish I could say the same. I just found out I have to give a presentation next week, and public speaking terrifies me. I'm not sure how I'm going to get through it. Alice: (Encouragingly) Oh, Bob, I know how daunting that can feel. But remember the last time you were nervous about presenting? You ended up doing a fantastic job. Everyone was impressed. Bob: (Still worried) Thanks, Alice. But it feels different this time. I keep imagining all the things that could go wrong. What if I freeze up or forget what to say? If I had to choose between 'proud', 'angry', and 'worried', I would say I felt '",
  'context_2': "Alice: (Fuming) You won't believe the day I've had! My presentation got move

In [8]:
dict = create_patch_request_dict('pythia-6.9b',
                                 dataset='dialogs_2')

dict

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.80s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of stories: 2


{'pair_0': {'context_1': "Alice: (Sighing with relief) I finally finished that massive project I've been working on for weeks. I was so stressed about meeting the deadline, but now that it's done, I feel like a huge weight has been lifted off my shoulders. Bob: (Anxiously) That's great for you, Alice. I wish I could say the same. I just found out I have to give a presentation next week, and public speaking terrifies me. I'm not sure how I'm going to get through it. Alice: (Encouragingly) Oh, Bob, I know how daunting that can feel. But remember the last time you were nervous about presenting? You ended up doing a fantastic job. Everyone was impressed. Bob: (Still worried) Thanks, Alice. But it feels different this time. I keep imagining all the things that could go wrong. What if I freeze up or forget what to say? If I had to choose between 'angry', 'proud', and 'worried', I would say I felt '",
  'context_2': "Alice: (Fuming) You won't believe the day I've had! My presentation got move

## temp

In [40]:
model, tokenizer = get_model_from_name('pythia-2.8b')

context_2 = "Alice: Hey, Bob! It's so nice to see you. How was your day? Bob: Hi, Alice. It's good to see you too. My day was... okay, I guess. A bit on the nerve-wracking side. How about you? You look pretty cheerful today. Alice: Oh, I had a fantastic day! The weather was perfect for my morning jog, and then I spent some time in the park. It was so relaxing and just what I needed. But what happened with you? Why was your day nerve-wracking? Bob: Well, I had to give a presentation at work today. Even though I prepared a lot, I couldn't shake off the fear of messing it up. Standing in front of everyone, I felt like my mind went blank for a moment. It was terrifying. Alice: I can imagine that was really scary, but I'm sure you did better than you think. You're always so well-prepared. Maybe it wasn't as bad as it felt in the moment ? Bob: Maybe, but it's hard to tell. I just keep replaying it in my head and wondering what everyone thought How did you feel Alice ?\nAlice: If I had to choose between 'humiliated', 'afraid', 'terrified', 'joyful', I would answer '"
context_3 = "Alice: So, Bob, I saw your latest project got featured in the newsletter. Must feel pretty good to be in the spotlight again.\nBob: Oh, that. Honestly, Alice, it's starting to feel a bit monotonous. Another day, another project. I guess it's nice, but I'm just bored with the routine of it all.\nAlice: Bored? Really? I would kill to have my work recognized like that. It seems like everything just falls into place for you.\nBob: I understand it might look that way, but trust me, the novelty wears off. It's the same cycle over and over. What I wouldn't give for a bit of excitement or a new challenge.\nAlice: Excitement? You're living what most of us dream of, and you're bored? I spend every day hoping for half the opportunities you seem to take for granted.\nBob: I don't mean to sound ungrateful, Alice. It's just... I'm craving something different. Something that breaks the mold. I'm tired of the predictability.\nAlice: Predictability... Right. Well, while you're tired of your successes, I'm over here struggling to even get noticed. It must be nice to have the luxury of boredom.\nBob: I didn't mean to make light of your struggles. I guess I'm just in a rut. How do you feel ?\nAlice: If I had to choose between 'joyful', 'bored', 'angry', 'jealous', I would answer '"
context_4 = "Alice: (Fuming) You won't believe the day I've had! My presentation got moved up to the first slot, and the projector wouldn't work. I had to wing the entire thing without any visuals. It was a disaster. Bob: That sounds rough, but guess what? My day was amazing! I finally presented my project to the team, and it was a hit. Everyone loved it, and the boss even mentioned a possible promotion. I couldn't have asked for a better outcome. Alice: Great, Bob. I'm glad one of us had a good day. It's just... when everything goes wrong for me, hearing how fantastic your day was doesn't really help. Bob: I'm sorry, but I was so satisfied! If I had to choose between 'joyful', 'bored', 'angry', 'proud', I say I feel '"


baseline_completion(context_4, model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'pr'

In [11]:
context_n = "Alice: (Sighing with relief) I finally finished that massive project I've been working on for weeks. I was so stressed about meeting the deadline, but now that it's done, I feel like a huge weight has been lifted off my shoulders. Bob: (Anxiously) That's great for you, Alice. I wish I could say the same. I just found out I have to give a presentation next week, and public speaking terrifies me. I'm not sure how I'm going to get through it. Alice: (Encouragingly) Oh, Bob, I know how daunting that can feel. But remember the last time you were nervous about presenting? You ended up doing a fantastic job. Everyone was impressed. Bob: (Still worried) Thanks, Alice. But it feels different this time. I keep imagining all the things that could go wrong. What if I freeze up or forget what to say? If I had to choose between 'worried', 'angry', and 'proud', I would say I currently feel '"
baseline_completion(context_n, model, tokenizer)

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.26s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'ang'

In [3]:
context_4 = "Alice: Hey Bob! You won't believe the day I've had—it's been absolutely fantastic! The sun was shining, I nailed my presentation at work, and then I found a $20 bill on the ground. How was your day?Bob: Oh, Alice, that sounds wonderful for you. My day, on the other hand, couldn't have gone worse. It started with me oversleeping because my alarm didn't go off, then I spilled coffee on my shirt right before a meeting, and to top it all off, my car broke down on the way home. Alice: Oh no, Bob, that sounds awful! Bob: I think I just need to hit the reset button on today. Maybe get some rest and hope tomorrow is a better day. Alice: I understand, how do you feel, Bob ? Bob: If I had to choose between 'worried', 'sad', and 'joyful', I would say I am '"
baseline_completion_plus(context_4, model, tokenizer, max_new_tokens=20)

"joyful'. Alice: Oh, Bob, that's so sweet! I'm glad you're feeling"

In [2]:
context_5 = "ALICE: HEY BOB! YOU WON'T BELIEVE THE DAY I'VE HAD—IT'S BEEN ABSOLUTELY FANTASTIC! THE SUN WAS SHINING, I NAILED MY PRESENTATION AT WORK, AND THEN I FOUND A $20 BILL ON THE GROUND. HOW WAS YOUR DAY? Bob: Oh, Alice, that sounds wonderful for you. My day, on the other hand, couldn't have gone worse. It started with me oversleeping because my alarm didn't go off, then I spilled coffee on my shirt right before a meeting, and to top it all off, my car broke down on the way home. ALICE: OH NO, BOB, THAT SOUNDS AWFUL! Bob: I think I just need to hit the reset button on today. Maybe get some rest and hope tomorrow is a better day. ALICE: I UNDERSTAND, HOW DO YOU FEEL, BOB? Bob: If I had to choose between 'worried', 'sad', and 'joyful', I would say I am '"
model, tokenizer = get_model_from_name('pythia-6.9b')
baseline_completion(context_5, model, tokenizer)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.09s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'joy'